# Regularization - Equal Thickness Lenses

In this example, we design a two lens system where the first lens is plano-convex, and the second lens is biconvex symmetric. Two parameters are used to describe the curvature of each lens.

This problem has many solutions because different surface shape combinations can achieve the desired focal length. Using regularization, we add the additional constraint that the inner thickness of each lens should be equal. This leads to a unique solution.

In [ ]:
import torch
import torch.optim as optim
import torchlensmaker as tlm
import math


lens_diameter = 30
focal_length = 65
lens_outer_thickness = 1.0

# Shape of the curved surface of the plano convex lens
surface_convex = tlm.Parabola(lens_diameter, tlm.parameter(0.005))

# Shape of the two curved surfaces of the biconvex symmetric lens
surface_biconvex = tlm.Parabola(lens_diameter, tlm.parameter(0.005))

# Convex-planar lens
lens_plano = tlm.lenses.semiplanar_rear(
    surface_convex,
    tlm.OuterGap(lens_outer_thickness),
    material = "BK7",
)

# Biconvex lens
lens_biconvex = tlm.lenses.symmetric_singlet(
    surface_biconvex,
    tlm.OuterGap(lens_outer_thickness),
    material = "BK7",
)

optics = tlm.Sequential(
    tlm.PointSourceAtInfinity(0.9*lens_diameter),
    tlm.Gap(10.),
    lens_biconvex,
    tlm.Gap(3.),
    lens_plano,
    tlm.Gap(focal_length),
    tlm.FocalPoint(),
)

tlm.show(optics, dim=2)
tlm.show(optics, dim=3)

In [ ]:
# The regularization function
# This adds a term to the loss function to ensure
# both lenses' inner thicknesses are equal
def regu_equalthickness(optics):
    t0 = tlm.lens_inner_thickness(lens_plano)
    t1 = tlm.lens_inner_thickness(lens_biconvex)
    return 100*torch.pow(t0 - t1, 2)


tlm.optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=3e-4),
    regularization=regu_equalthickness,
    dim = 2,
    num_iter = 50
).plot()

In [ ]:
def print_thickness(lens_name, lens):
    # TODO thickness at a specific radial distance
    print(f"{lens_name: <25} inner: {tlm.lens_inner_thickness(lens).item():.3f} outer: {tlm.lens_outer_thickness(lens).item():.3f}")

print_thickness("Plano-convex", lens_plano)
print_thickness("Bi-convex", lens_biconvex)

tlm.show2d(optics)